In [1]:
import boto3
import pandas as pd
from io import StringIO  # python3; python2: BytesIO
import time

In [ ]:
AWS_ACCESS_KEY = "---"
AWS_SECRET_KEY = "---"
AWS_REGION = "us-west-2"
SCHEMA_NAME = "data-wearhousing-db"
S3_STAGING_DIR = "s3://s3-data-wearhousing/output/"
S3_BUCKET_NAME = "s3-data-wearhousing"
S3_OUTPUT_DIRECTORY = "output"

In [8]:
athena_client = boto3.client(
    "athena",
    aws_access_key_id=AWS_ACCESS_KEY,
    aws_secret_access_key=AWS_SECRET_KEY,
    region_name=AWS_REGION,
)


In [9]:
athena_client

In [10]:
Dict = {}

def download_and_load_query_results(
    client: boto3.client, query_response: Dict
) -> pd.DataFrame:
    while True:
        try:
            client.get_query_results(
                QueryExecutionId=query_response["QueryExecutionId"]
            )
            break
        except Exception as err:
            if "not yet finished" in str(err):
                time.sleep(0.001)
            else:
                raise err
    temp_file_location: str = "athena_query_results.csv"
    s3_client = boto3.client(
        "s3",
        aws_access_key_id=AWS_ACCESS_KEY,
        aws_secret_access_key=AWS_SECRET_KEY,
        region_name=AWS_REGION,
    )
    s3_client.download_file(
        S3_BUCKET_NAME,
        f"{S3_OUTPUT_DIRECTORY}/{query_response['QueryExecutionId']}.csv",
        temp_file_location,
    )
    return pd.read_csv(temp_file_location)


In [31]:
response = athena_client.start_query_execution(
QueryString="SELECT * FROM enigma_jhud",
QueryExecutionContext={"Database": SCHEMA_NAME},
ResultConfiguration={ "OutputLocation": S3_STAGING_DIR,
"EncryptionConfiguration": {"EncryptionOption": "SSE_S3"}, },
)

enigma_jhud = download_and_load_query_results(athena_client, response)

enigma_jhud.head()
# 11

,fips,admin2,province_state,country_region,last_update,latitude,longitude,confirmed,deaths,recovered,active,combined_key
0,NaN,NaN,Anhui,China,2020-01-22T17:00:00,31.826,117.226,1.0,NaN,NaN,NaN,"""Anhui"
1,NaN,NaN,Beijing,China,2020-01-22T17:00:00,40.182,116.414,14.0,NaN,NaN,NaN,"""Beijing"
2,NaN,NaN,Chongqing,China,2020-01-22T17:00:00,30.057,107.874,6.0,NaN,NaN,NaN,"""Chongqing"
3,NaN,NaN,Fujian,China,2020-01-22T17:00:00,26.079,117.987,1.0,NaN,NaN,NaN,"""Fujian"
4,NaN,NaN,Gansu,China,2020-01-22T17:00:00,36.061,103.834,NaN,NaN,NaN,NaN,"""Gansu"


In [12]:
enigma_jhud.shape

(10002, 12)

In [32]:
response = athena_client.start_query_execution(
QueryString="SELECT * FROM country",
QueryExecutionContext={"Database": SCHEMA_NAME},
ResultConfiguration={ "OutputLocation": S3_STAGING_DIR,
"EncryptionConfiguration": {"EncryptionOption": "SSE_S3"}, },
)

country = download_and_load_query_results(athena_client, response)

country.head()
# 11

,date,county,state,fips,cases,deaths
0,1/21/2020,Snohomish,Washington,53061.0,1,0
1,1/22/2020,Snohomish,Washington,53061.0,1,0
2,1/23/2020,Snohomish,Washington,53061.0,1,0
3,1/24/2020,Cook,Illinois,17031.0,1,0
4,1/24/2020,Snohomish,Washington,53061.0,1,0


In [33]:
response = athena_client.start_query_execution(
QueryString="SELECT * FROM countypopulation",
QueryExecutionContext={"Database": SCHEMA_NAME},
ResultConfiguration={ "OutputLocation": S3_STAGING_DIR,
"EncryptionConfiguration": {"EncryptionOption": "SSE_S3"}, },
)

countypopulation = download_and_load_query_results(athena_client, response)

countypopulation.head()
# 11

,id,id2,county,state,population estimate 2018
0,0500000US01001,1001,Autauga,Alabama,55601
1,0500000US01003,1003,Baldwin,Alabama,218022
2,0500000US01005,1005,Barbour,Alabama,24881
3,0500000US01007,1007,Bibb,Alabama,22400
4,0500000US01009,1009,Blount,Alabama,57840


In [34]:
response = athena_client.start_query_execution(
QueryString="SELECT * FROM rearc_usa_hospital_beds",
QueryExecutionContext={"Database": SCHEMA_NAME},
ResultConfiguration={ "OutputLocation": S3_STAGING_DIR,
"EncryptionConfiguration": {"EncryptionOption": "SSE_S3"}, },
)

rearc_usa_hospital_beds = download_and_load_query_results(athena_client, response)

rearc_usa_hospital_beds.head()
# 11

,objectid,hospital_name,hospital_type,hq_address,hq_address1,hq_city,hq_state,hq_zip_code,county_name,state_name,...,num_licensed_beds,num_staffed_beds,num_icu_beds,adult_icu_beds,pedi_icu_beds,bed_utilization,avg_ventilator_usage,potential_increase_in_bed_capac,latitude,longtitude
0,1,Phoenix VA Health Care System (AKA Carl T Hayd...,VA Hospital,650 E Indian School Rd,NaN,Phoenix,AZ,85012,Maricopa,Arizona,...,129.0,129.0,0,0,NaN,NaN,0.0,0,33.495498,-112.066157
1,2,Southern Arizona VA Health Care System,VA Hospital,3601 S 6th Ave,NaN,Tucson,AZ,85723,Pima,Arizona,...,295.0,295.0,2,2,NaN,NaN,2.0,0,32.181263,-110.965885
2,3,VA Central California Health Care System,VA Hospital,2615 E Clinton Ave,NaN,Fresno,CA,93703,Fresno,California,...,57.0,57.0,2,2,NaN,NaN,2.0,0,36.773324,-119.779742
3,4,VA Connecticut Healthcare System - West Haven ...,VA Hospital,950 Campbell Ave,NaN,West Haven,CT,6516,New Haven,Connecticut,...,216.0,216.0,1,1,NaN,NaN,2.0,0,41.284400,-72.957610
4,5,Wilmington VA Medical Center,VA Hospital,1601 Kirkwood Hwy,NaN,Wilmington,DE,19805,New Castle,Delaware,...,60.0,60.0,0,0,NaN,NaN,1.0,0,39.740206,-75.606532


In [35]:
response = athena_client.start_query_execution(
QueryString="SELECT * FROM state",
QueryExecutionContext={"Database": SCHEMA_NAME},
ResultConfiguration={ "OutputLocation": S3_STAGING_DIR,
"EncryptionConfiguration": {"EncryptionOption": "SSE_S3"}, },
)

state = download_and_load_query_results(athena_client, response)

state.head()
# 11

,date,state,fips,cases,deaths
0,1/21/2020,Washington,53,1,0
1,1/22/2020,Washington,53,1,0
2,1/23/2020,Washington,53,1,0
3,1/24/2020,Illinois,17,1,0
4,1/24/2020,Washington,53,1,0


In [36]:
response = athena_client.start_query_execution(
QueryString="SELECT * FROM states_daily",
QueryExecutionContext={"Database": SCHEMA_NAME},
ResultConfiguration={ "OutputLocation": S3_STAGING_DIR,
"EncryptionConfiguration": {"EncryptionOption": "SSE_S3"}, },
)

states_daily = download_and_load_query_results(athena_client, response)

states_daily.head()
# 11

,date,state,positive,probablecases,negative,pending,totaltestresultssource,totaltestresults,hospitalizedcurrently,hospitalizedcumulative,...,dataqualitygrade,deathincrease,hospitalizedincrease,hash,commercialscore,negativeregularscore,negativescore,positivescore,score,grade
0,20210307,AK,56886,NaN,NaN,NaN,totalTestsViral,1731628,33.0,1293.0,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
1,20210307,AL,499819,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788,494.0,45976.0,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2,20210307,AR,324818,69092.0,2480716.0,NaN,totalTestsViral,2736442,335.0,14926.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
3,20210307,AS,0,NaN,2140.0,NaN,totalTestsViral,2140,NaN,NaN,...,NaN,0,0,96d23f888c995b9a7f3b4b864de6414f45c728ff,0,0,0,0,0,NaN
4,20210307,AZ,826454,56519.0,3073010.0,NaN,totalTestsViral,7908105,963.0,57907.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


In [37]:
response = athena_client.start_query_execution(
QueryString="SELECT * FROM statesabv",
QueryExecutionContext={"Database": SCHEMA_NAME},
ResultConfiguration={ "OutputLocation": S3_STAGING_DIR,
"EncryptionConfiguration": {"EncryptionOption": "SSE_S3"}, },
)

statesabv = download_and_load_query_results(athena_client, response)

statesabv.head()
# 11

,col0,col1
0,State,Abbreviation
1,Alabama,AL
2,Alaska,AK
3,Arizona,AZ
4,Arkansas,AR


In [38]:
response = athena_client.start_query_execution(
QueryString="SELECT * FROM us_daily",
QueryExecutionContext={"Database": SCHEMA_NAME},
ResultConfiguration={ "OutputLocation": S3_STAGING_DIR,
"EncryptionConfiguration": {"EncryptionOption": "SSE_S3"}, },
)

us_daily = download_and_load_query_results(athena_client, response)

us_daily.head()
# 11

,date,states,positive,negative,pending,hospitalizedcurrently,hospitalizedcumulative,inicucurrently,inicucumulative,onventilatorcurrently,...,lastmodified,recovered,total,posneg,deathincrease,hospitalizedincrease,negativeincrease,positiveincrease,totaltestresultsincrease,hash
0,20210307,56,28755524.0,74579770.0,11808.0,40212.0,878613.0,8137.0,45475.0,2801.0,...,2021-03-07T24:00:00Z,NaN,0,0,839,726,130414,41265,1156241,8b26839690cd05c0cef69cb9ed85641a76b5e78e
1,20210306,56,28714259.0,74449356.0,11783.0,41401.0,877887.0,8409.0,45453.0,2811.0,...,2021-03-06T24:00:00Z,NaN,0,0,1674,503,142201,59620,1409138,d0c0482ea549c9d5c04a7c86acb6fc6a8095a592
2,20210305,56,28654639.0,74307155.0,12213.0,42541.0,877384.0,8634.0,45373.0,2889.0,...,2021-03-05T24:00:00Z,NaN,0,0,2221,2781,271917,68787,1744417,a35ea4289cec4bb55c9f29ae04ec0fd5ac4e0222
3,20210304,56,28585852.0,74035238.0,12405.0,44172.0,874603.0,8970.0,45293.0,2973.0,...,2021-03-04T24:00:00Z,NaN,0,0,1743,1530,177957,65487,1590984,a19ad6379a653834cbda3093791ad2c3b9fab5ff
4,20210303,56,28520365.0,73857281.0,11778.0,45462.0,873073.0,9359.0,45214.0,3094.0,...,2021-03-03T24:00:00Z,NaN,0,0,2449,2172,267001,66836,1406795,9e1d2afda1b0ec243060d6f68a7134d011c0cb2a


In [39]:
response = athena_client.start_query_execution(
QueryString="SELECT * FROM us_total_latest",
QueryExecutionContext={"Database": SCHEMA_NAME},
ResultConfiguration={ "OutputLocation": S3_STAGING_DIR,
"EncryptionConfiguration": {"EncryptionOption": "SSE_S3"}, },
)

us_total_latest = download_and_load_query_results(athena_client, response)

us_total_latest.head()
# 11

,positive,negative,pending,hospitalizedcurrently,hospitalizedcumulative,inicucurrently,inicucumulative,onventilatorcurrently,onventilatorcumulative,recovered,hash,lastmodified,death,hospitalized,total,totaltestresults,posneg,notes
0,1061101,5170081,2775,53793,111955,9486,4192,4712,373,153947,95064ba29ccbc20dbec397033dfe4b1f45137c99,2020-05-01T09:12:31.891Z,57266,111955,6233957,6231182,6231182,"""NOTE: """"total"""""


In [40]:
# fips		province_state	country_region	last_update	latitude	longitude	confirmed	deaths	recovered	active	combined_key
factCovid_1= enigma_jhud[['fips', 'province_state', 'country_region', 'confirmed', 'deaths', 'recovered', 'active']]
factCovid_2= states_daily[['fips', 'date', 'positive', 'negative', 'hospitalizedcurrently','hospitalized','hospitalizeddischarged']]
factCovid = pd.merge(factCovid_1, factCovid_2, on='fips', how='inner')

In [41]:
factCovid.head()

,fips,province_state,country_region,confirmed,deaths,recovered,active,date,positive,negative,hospitalizedcurrently,hospitalized,hospitalizeddischarged
0,72.0,Puerto Rico,US,3.0,0.0,0.0,NaN,20210307,101327,305972.0,147.0,NaN,NaN
1,72.0,Puerto Rico,US,3.0,0.0,0.0,NaN,20210306,101327,305972.0,147.0,NaN,NaN
2,72.0,Puerto Rico,US,3.0,0.0,0.0,NaN,20210305,101066,305972.0,136.0,NaN,NaN
3,72.0,Puerto Rico,US,3.0,0.0,0.0,NaN,20210304,100867,305972.0,171.0,NaN,NaN
4,72.0,Puerto Rico,US,3.0,0.0,0.0,NaN,20210303,100765,305972.0,169.0,NaN,NaN


In [43]:
dimRegion_1 = enigma_jhud [['fips', 'province_state', 'country_region', 'latitude', 'longitude']]
dimRegion_2 = country [['fips', 'county', 'state']]
dimRegion = pd.merge(dimRegion_1, dimRegion_2, on='fips', how='inner')

In [44]:
dimHospital = rearc_usa_hospital_beds[['fips', 'state_name', 'latitude', 'longtitude', 'hq_address', 'hospital_name', 'hospital_type', 'hq_city', 'hq_state']]


In [45]:
dimDate = states_daily[['fips', 'date']]
dimDate.head()


,fips,date
0,2,20210307
1,1,20210307
2,5,20210307
3,60,20210307
4,4,20210307


In [46]:
dimDate['date'] = pd.to_datetime(dimDate['date'], format='%Y%m%d')


C:\Users\MUHAMMAD NOMAN\AppData\Local\Temp\ipykernel_5316\4254038327.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dimDate['date'] = pd.to_datetime(dimDate['date'], format='%Y%m%d')


In [47]:
dimDate.loc[:, 'date'] = pd.to_datetime(dimDate['date'], format='%Y%m%d')

In [48]:
dimDate['month'] = dimDate['date'].dt.month
dimDate['year'] = dimDate['date'].dt.year
dimDate['day_of_week'] = dimDate['date'].dt.dayofweek


C:\Users\MUHAMMAD NOMAN\AppData\Local\Temp\ipykernel_5316\977630067.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dimDate['month'] = dimDate['date'].dt.month
C:\Users\MUHAMMAD NOMAN\AppData\Local\Temp\ipykernel_5316\977630067.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dimDate['year'] = dimDate['date'].dt.year
C:\Users\MUHAMMAD NOMAN\AppData\Local\Temp\ipykernel_5316\977630067.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_i

In [49]:
# Convert to datetime format
dimDate['date'] = pd.to_datetime(dimDate['date'], format='%m/%d/%Y')

# Format as YYYY-MM-DD
dimDate['date'] =dimDate['date'].dt.strftime('%Y-%m-%d')
dimDate.head()

C:\Users\MUHAMMAD NOMAN\AppData\Local\Temp\ipykernel_5316\1307554633.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dimDate['date'] = pd.to_datetime(dimDate['date'], format='%m/%d/%Y')
C:\Users\MUHAMMAD NOMAN\AppData\Local\Temp\ipykernel_5316\1307554633.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dimDate['date'] =dimDate['date'].dt.strftime('%Y-%m-%d')


,fips,date,month,year,day_of_week
0,2,2021-03-07,3,2021,6
1,1,2021-03-07,3,2021,6
2,5,2021-03-07,3,2021,6
3,60,2021-03-07,3,2021,6
4,4,2021-03-07,3,2021,6


In [50]:
dimDate.head()

,fips,date,month,year,day_of_week
0,2,2021-03-07,3,2021,6
1,1,2021-03-07,3,2021,6
2,5,2021-03-07,3,2021,6
3,60,2021-03-07,3,2021,6
4,4,2021-03-07,3,2021,6


In [51]:
output_bucket = 's3-data-wearhousing-output--um' # already created on S3

csv_buffer = StringIO()
csv_buffer

In [52]:
csv_buffer = StringIO()
dimDate.to_csv(csv_buffer,index=False)

s3_resource =boto3.resource('s3')
s3_resource.Object(output_bucket, 'dimDate.csv').put(Body=csv_buffer.getvalue())


{'ResponseMetadata': {'RequestId': '25WN74R962G964J2',
  'HostId': 'UwiC8QLRpe6EGHkC1fG1ZiSDSm+iBYVaP7FCkKPrNSZ2E4TQreCi22hinOw5lMKgelGqx98MA8yorjNK5KheCg==',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'UwiC8QLRpe6EGHkC1fG1ZiSDSm+iBYVaP7FCkKPrNSZ2E4TQreCi22hinOw5lMKgelGqx98MA8yorjNK5KheCg==',
   'x-amz-request-id': '25WN74R962G964J2',
   'date': 'Sat, 30 Nov 2024 17:49:12 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"1c5698d5ecd349bc6e1a02362be9d8fe"',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'ETag': '"1c5698d5ecd349bc6e1a02362be9d8fe"',
 'ServerSideEncryption': 'AES256'}

In [53]:
csv_buffer = StringIO()
dimRegion.to_csv(csv_buffer,index=False)

s3_resource =boto3.resource('s3')
s3_resource.Object(output_bucket, 'dimRegion.csv').put(Body=csv_buffer.getvalue())


{'ResponseMetadata': {'RequestId': 'NS9MEAS7PC42KTES',
  'HostId': 'VQXSy47TiU5wuzGqAAhqTBbvqy71kRaIfU5wKj7oK6aKYhQ3t9/kQy/Qb9Tih1ESnZ0U9/FfBSU=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'VQXSy47TiU5wuzGqAAhqTBbvqy71kRaIfU5wKj7oK6aKYhQ3t9/kQy/Qb9Tih1ESnZ0U9/FfBSU=',
   'x-amz-request-id': 'NS9MEAS7PC42KTES',
   'date': 'Sat, 30 Nov 2024 17:56:57 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"9f9060e223b59c02f1c9053bb8729109"',
   'content-length': '0',
   'server': 'AmazonS3',
   'connection': 'close'},
  'RetryAttempts': 1},
 'ETag': '"9f9060e223b59c02f1c9053bb8729109"',
 'ServerSideEncryption': 'AES256'}

In [54]:
csv_buffer = StringIO()
dimHospital.to_csv(csv_buffer,index=False)

s3_resource =boto3.resource('s3')
s3_resource.Object(output_bucket, 'dimHospital.csv').put(Body=csv_buffer.getvalue())


{'ResponseMetadata': {'RequestId': '469AE2ERRYYP1FCW',
  'HostId': 'bPCEIV+PCH33WTjHLYYfQWv5RoJwwIQJmtCc1CYHw5GvkoBtr9bsgpPSG2z/9VmYY4nmWTs2zMr2hBYWP0CVPw==',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'bPCEIV+PCH33WTjHLYYfQWv5RoJwwIQJmtCc1CYHw5GvkoBtr9bsgpPSG2z/9VmYY4nmWTs2zMr2hBYWP0CVPw==',
   'x-amz-request-id': '469AE2ERRYYP1FCW',
   'date': 'Sat, 30 Nov 2024 18:04:13 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"d7541fc3f907bdc43355ce9051a0ac5e"',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'ETag': '"d7541fc3f907bdc43355ce9051a0ac5e"',
 'ServerSideEncryption': 'AES256'}

In [55]:
csv_buffer = StringIO()
factCovid.to_csv(csv_buffer,index=False)

s3_resource =boto3.resource('s3')
s3_resource.Object(output_bucket, 'factCovid.csv').put(Body=csv_buffer.getvalue())

{'ResponseMetadata': {'RequestId': 'FFGDDKABZXV47GPE',
  'HostId': 'WCHujKjhxb2KgYdkbJAgYvIdQWJEloKezSN5MUk/cl3kXXGoTHgo62IEDRZuXoIMryfPeM9hWVhDN0yF3TG6QQ==',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'WCHujKjhxb2KgYdkbJAgYvIdQWJEloKezSN5MUk/cl3kXXGoTHgo62IEDRZuXoIMryfPeM9hWVhDN0yF3TG6QQ==',
   'x-amz-request-id': 'FFGDDKABZXV47GPE',
   'date': 'Sat, 30 Nov 2024 18:05:25 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"f01455d2138fda58e867126fbb10a7e9"',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'ETag': '"f01455d2138fda58e867126fbb10a7e9"',
 'ServerSideEncryption': 'AES256'}

In [ ]:
import pandas as pd

# Assuming dimDate, factCovid, dimRegion, and dimHospital are predefined DataFrames

dimDatesql = pd.io.sql.get_schema(dimDate.reset_index(), 'dimDate')
print(''.join(dimDatesql))

factCovidsql = pd.io.sql.get_schema(factCovid.reset_index(), 'factCovid')
print(''.join(factCovidsql))

dimRegionsql = pd.io.sql.get_schema(dimRegion.reset_index(), 'dimRegion')
print(''.join(dimRegionsql))

dimHospitalsql = pd.io.sql.get_schema(dimHospital.reset_index(), 'dimHospital')
print(''.join(dimHospitalsql))


In [ ]:
pip install redshift-connector

In [ ]:
import redshift_connector
import logging

# logging.basicConfig(level=logging.DEBUG)

try:
    conn = redshift_connector.connect(
        host='redshift-cluster-2.czfgt1tbsuck.us-east-2.redshift.amazonaws.com',
        database='dev',
        user='---',
        password='-----',
        ssl=True,
        timeout=60  # Increase the timeout if the connection takes longer
    )
    print(conn)
except redshift_connector.Error as e:
    print(f"Error: {e}")


In [ ]:
# Enabling autocommit
conn.autocommit = True

# Creating a cursor object to interact with the database
cursor = conn.cursor()

# Creating the dimDate table
cursor.execute("""
CREATE TABLE "dimDate" (
    "fips" INTEGER,
    "date" TIMESTAMP,
    "year" INTEGER,
    "month" INTEGER,
    "day_of_week" INTEGER
)
""")


cursor.execute("""
CREATE TABLE "dimHospital" (
  "fips" REAL,
  "state_name" TEXT,
  "latitude" REAL,
  "longitude" REAL,
  "hq_address" TEXT,
  "hospital_name" TEXT,
  "hospital_type" TEXT,
  "hq_city" TEXT,
  "hq_state" TEXT
)
""")



In [ ]:
# Enabling autocommit
conn.autocommit = True

# Creating a cursor object to interact with the database
cursor = conn.cursor()

# Creating the dimDate table
cursor.execute("""
CREATE TABLE "factCovid" (
  "fips" REAL,
  "province_state" TEXT,
  "country_region" TEXT,
  "confirmed" REAL,
  "deaths" REAL,
  "recovered" REAL,
  "active" REAL,
  "date" INTEGER,
  "positive" INTEGER,
  "negative" REAL,
  "hospitalized_currently" REAL,
  "hospitalized" REAL,
  "hospitalized_discharged" REAL
)
""")

# Creating the dimRegion table
cursor.execute("""
CREATE TABLE "dimRegion" (
  "fips" REAL,
  "province_state" TEXT,
  "country_region" TEXT,
  "latitude" REAL,
  "longitude" REAL,
  "county" TEXT,
  "state" TEXT
)
""")



In [ ]:
cursor.execute("""
copy factcovid from 's3://aws-session-datawarehousing-output/factCovid.csv'
credentials 'aws_iam_role=arn:aws:iam::975050334693:role/service-role/AmazonRedshift-CommandsAccessRole-20240625T125922'
delimiter ','
region 'us-east-2'
IGNOREHEADER 1;
""")

cursor.execute("""
copy dimregion from 's3://aws-session-datawarehousing-output/dimRegion.csv'
credentials 'aws_iam_role=arn:aws:iam::975050334693:role/service-role/AmazonRedshift-CommandsAccessRole-20240625T125922'
delimiter ','
region 'us-east-2'
IGNOREHEADER 1;
""")


cursor.execute("""
copy dimhospital from 's3://aws-session-datawarehousing-output/dimHospital.csv'
credentials 'aws_iam_role=arn:aws:iam::975050334693:role/service-role/AmazonRedshift-CommandsAccessRole-20240625T125922'
delimiter ','
region 'us-east-2'
IGNOREHEADER 1;
""")


cursor.execute("""
copy dimdate from 's3://aws-session-datawarehousing-output/dimDate.csv'
credentials 'aws_iam_role=arn:aws:iam::975050334693:role/service-role/AmazonRedshift-CommandsAccessRole-20240625T125922'
delimiter ','
region 'us-east-2'
IGNOREHEADER 1;
""")